In [ ]:
#####################################
# This creates figure(s)
# LENS_fig{1,2}.png 
# in paper
#####################################
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import copy
import importlib
import get_lens1_rcp85 as lens1
import sst_biases_2018pub as sstbias
import get_hadley as Had
import PlotUtil as Pu
import ts_w_trx_joint as ttJ



importlib.reload( sstbias )
importlib.reload( lens1 )
importlib.reload( Had )
importlib.reload( Pu )
importlib.reload( ttJ )


In [ ]:
###
# Get Hadley SSTs
###
hadsst,lfrac_had,dates_had,lat_had,lon_had = Had.hadley1x1( limit_dates_for_pub=True)

In [ ]:
print(np.shape(dates_had))
print(dates_had[0,0],dates_had[-1,-1])

In [ ]:
monthly_mean_bias,lat_sst,lon_sst = sstbias.original()

In [ ]:
# Get Future SSTs from LENS
# This also gets a CESM1 CMIP5 RCP8.5 run
# which is 
tsFu,landf,lat,lon = lens1.FullTS()
tsFuC=copy.deepcopy( tsFu )

In [ ]:
tsPD,landfPD,latPD,lonPD = lens1.PresentDayTS()
tsPDC=copy.deepcopy( tsPD )

In [ ]:
###
# Calculate bias corretced TS field using mean monthly biases

nens,nt,ny,nx = np.shape( tsFu )
nYr =  nt//12 

tsFuC = tsFuC.reshape( nens ,nYr, 12, ny, nx )
tsFuR = np.zeros( (nens ,nYr, 12, ny, nx ) )

for nen in np.arange( nens ):
    for iy in np.arange( nYr ):
        for im in np.arange( 12 ):
            tsFuR[nen,iy,im,:,:] = tsFuC[nen,iy,im,:,:] - monthly_mean_bias[im,:,:]

In [ ]:
###
# Take time mean, i.e. over Year,Month '_aYM'.
# Then find time-mean deviation of each ens member 
# from ensemable mean '_aYM_Ep'

tsFuR_aYM    = np.average( np.average( tsFuR , axis= 1 ), axis= 1 )
tsFuR_aYM_Ep = tsFuR_aYM - tsFuR_aYM.mean( axis=0 )
tsFuC_aYM    = np.average( np.average( tsFuC , axis= 1 ), axis= 1 )
tsFuC_aYM_Ep = tsFuC_aYM - tsFuC_aYM.mean( axis=0 )

In [ ]:
nens,nt,ny,nx = np.shape( tsPD )
nYr =  nt//12 

tsPDC = tsPDC.reshape( nens ,nYr, 12, ny, nx )
tsPDR = np.zeros( (nens ,nYr, 12, ny, nx ) )

for nen in np.arange( nens ):
    for iy in np.arange( nYr ):
        for im in np.arange( 12 ):
            tsPDR[nen,iy,im,:,:] = tsPDC[nen,iy,im,:,:] - monthly_mean_bias[im,:,:]

In [ ]:
###
# Take time mean, i.e. over Year,Month '_aYM'.
# Then find time-mean deviation of each ens member 
# from ensemable mean '_aYM_Ep'

tsPDR_aYM    = np.average( np.average( tsPDR , axis= 1 ), axis= 1 )
tsPDR_aYM_Ep = tsPDR_aYM - tsPDR_aYM.mean( axis=0 )
tsPDC_aYM    = np.average( np.average( tsPDC , axis= 1 ), axis= 1 )
tsPDC_aYM_Ep = tsPDC_aYM - tsPDC_aYM.mean( axis=0 )

In [ ]:
#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

ensN=[0,-1]

fig = plt.figure(figsize=(20, 12.5))
sstlv=np.linspace(273,308,num=36)
for nplo in np.arange(2):

    nxplo=2 
    nyplo=1
    nen=ensN[nplo]
    
    Axes = Pu.axes_def(n=nplo+1,nxplo=nxplo,nyplo=nyplo ) 

    print( Axes )
    ax = fig.add_axes( Axes , projection=projPC)
    #ax=fig.add_subplot(6,5,nen  , projection=projPC )

    co=ax.contourf(lon,lat, tsFuC_aYM[nen,:,:] ,levels=sstlv,cmap='jet' , transform=data_crs )
    cl=ax.contour( lon,lat,tsFuC_aYM[nen,:,:] ,levels=sstlv,  transform=data_crs )
    #ax.coastlines(resolution='110m',color='black')
    ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
    #ax.set_ylim(60,130)
    #ax.set_extent( 
    ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
    ax.set_title( '#'+str(nen) )

cax = plt.axes([0.0, 0.8, 0.9, 0.02])
plt.colorbar(co,cax=cax,orientation='horizontal')


In [ ]:

#        j   f   m   a   m   j   j   a   s   o   n   d  
monwgt=[31.,28.,31.,30.,31.,30.,31.,31.,30.,31.,30.,31.]
latwgt=np.cos( lat*np.pi/180. )
plt.plot(latwgt)
print(np.shape(tsFuC))
tsFuCa = np.average( tsFuC , axis=4 )
print(np.shape(tsFuCa))
tsFuCa = np.average( tsFuCa , weights=latwgt, axis=3 )
print(np.shape(tsFuCa))
tsFuCa = np.average( tsFuCa , weights=monwgt, axis=2 )
print(np.shape(tsFuCa))


In [ ]:
print(np.shape(tsPDC))
tsPDCa = np.average( tsPDC , axis=4 )
print(np.shape(tsPDCa))
tsPDCa = np.average( tsPDCa , weights=latwgt, axis=3 )
print(np.shape(tsPDCa))
tsPDCa = np.average( tsPDCa , weights=monwgt, axis=2 )
print(np.shape(tsPDCa))


In [ ]:
pub_ens=[8,11,21,24,29]
tsPD_avg =np.average(tsPDCa) 

FuYear = np.arange(31)+2070
for e in np.arange( start=1,stop=nens ):
    plt.plot( FuYear, tsFuCa[e,:] - tsPD_avg ,color='cyan')

e=34
plt.plot( FuYear, tsFuCa[e,:] - tsPD_avg,color='black' ,linewidth=2)
e=0
plt.plot( FuYear, tsFuCa[e,:] - tsPD_avg,color='red' )

for e in pub_ens:
    plt.plot( FuYear, tsFuCa[e,:] - tsPD_avg,color='green' )

plt.ylabel( "Warming ($^\circ$C)" )
plt.title( "Global mean warning in LENS1" )
#for e in np.arange( start=0,stop=nensPD ):
#    plt.plot( tsPDCa[e,:] ,color='cyan')
    


In [ ]:
#############################
# Above Figure used in paper
#############################

In [ ]:
### STOP HERE FOR FIRST LOOK


In [ ]:
### STOP HERE FOR FIRST LOOK


In [ ]:
#### CONTINUE ???

In [ ]:
print( hadsst.shape )
hadmean = (hadsst.mean(axis=0)).mean(axis=0)
print(hadmean.shape)

In [ ]:
#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

ensN=[0,-1]

fig = plt.figure(figsize=(30, 12.5))
sstlv=np.linspace(273,308,num=36)
wrmlv=np.linspace(-1,5,num=13)
wrmlv=np.linspace(-10,10,num=21)

nxplo=3 
nyplo=1

tsPDC_aYME = tsPDC_aYM.mean(axis=0)

warming = tsFuC_aYM[-1,:,:] - tsPDC_aYME
warming2 = tsFuR_aYM[-1,:,:] - hadmean

nplo=1
Axes = Pu.axes_def(n=3,nxplo=nxplo,nyplo=nyplo ) 

ax = fig.add_axes( Axes , projection=projPC)
co=ax.contourf(lon,lat, warming2 ,levels=wrmlv,cmap='bwr' , transform=data_crs )
cl=ax.contour( lon,lat,warming2 ,levels=wrmlv,  transform=data_crs )
#ax.coastlines(resolution='110m',color='black')
ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
#ax.set_ylim(60,130)
#ax.set_extent( 
ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
#ax.set_title( '#'+str(nen) )
plt.colorbar( co ,shrink=.15)


Axes = Pu.axes_def(n=2,nxplo=nxplo,nyplo=nyplo ) 

ax = fig.add_axes( Axes , projection=projPC)
#ax=fig.add_subplot(6,5,nen  , projection=projPC )

#co=ax.contourf(lon,lat, tsPDav ,levels=sstlv,cmap='jet' , transform=data_crs )
#cl=ax.contour( lon,lat,tsPDav ,levels=sstlv,  transform=data_crs )
co=ax.contourf(lon,lat, warming ,levels=wrmlv,cmap='bwr' , transform=data_crs )
cl=ax.contour( lon,lat,warming ,levels=wrmlv,  transform=data_crs )
#ax.coastlines(resolution='110m',color='black')
ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
#ax.set_ylim(60,130)
#ax.set_extent( 
ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
#ax.set_title( '#'+str(nen) )
plt.colorbar( co ,shrink=.15)





Axes = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
ax = fig.add_axes( Axes , projection=projPC)

co=ax.contourf(lon,lat, tsFuR_aYM[-1,:,:] ,levels=sstlv,cmap='jet' , transform=data_crs )
cl=ax.contour( lon,lat,tsFuR_aYM[-1,:,:] ,levels=sstlv,  transform=data_crs )
#ax.coastlines(resolution='110m',color='black')
ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
#ax.set_ylim(60,130)
#ax.set_extent( 
ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
#ax.set_title( '#'+str(nen) )
plt.colorbar( co ,shrink=.15)


#cax = plt.axes([0.0, 0.8, 0.9, 0.02])
#plt.colorbar(co,cax=cax,orientation='horizontal')


In [ ]:
warming2 = tsFuR_aYM[-1,:,:] - hadmean
latwgt=np.cos( lat*np.pi/180. )
latwgt=np.where(np.abs(lat)<30,latwgt,0)
plt.plot(lat,latwgt)
warming_av=np.average( np.average(warming2,axis=1), weights=latwgt, axis=0)
print(warming_av)
warming=warming2-warming_av

In [ ]:
###
"""
Note, below when adding panel labels with ax.text:
f"{chr(97 + n)})"
gets lower case letters starting with 'a' at chr(97).
Uppercase starts at chr(65)
"""
pub_ens_x=[0,8,11,21,24,29,-1]
SSTlab = ['SST1' , 'SST2' , 'SST3' , 'SST4' , 'SST5' , 'SST6' , 'SST7 (bias corrected LENS1 ens. mean - Hadley SSTs)' ]
#panlab = [ 
projMap  = ccrs.PlateCarree(central_longitude=180.)
projData = ccrs.PlateCarree()

fig = plt.figure(figsize=(20, 8))
cmap='bwr'
nxplo,nyplo=3,3
for n in np.arange(start=0,stop=7):
    nen=pub_ens_x[n]
    if (nen != -1):
        Axes = Pu.axes_def(n=n+4,nxplo=nxplo,nyplo=nyplo ) 
        print(Axes)
        ax = fig.add_axes( Axes , projection=projMap)
        sstlv=np.linspace(-.5,.5,num=11)
        co=ax.contourf(lon,lat,tsFuR_aYM_Ep[nen,:,:] ,levels=sstlv,cmap=cmap,transform=projData,extend='both')
        cl=ax.contour( lon,lat,tsFuR_aYM_Ep[nen,:,:] ,levels=sstlv ,transform=projData, colors='black')
        ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
        ax.text(-0.1, 1.1, f"{chr(97 +n+1)})", transform=ax.transAxes,
            fontsize=20, fontweight='bold', va='top')
        #fig.text(Axes[0], Axes[1]+Axes[3], "Z)",
        #    fontsize=20, fontweight='bold', va='top')
        ax.set_title( SSTlab[n]+' - SST7'  )
    else:
        #sstlv=np.linspace(272,308,num=19)

        Axes = Pu.axes_def(n=1,nxplo=nxplo,nyplo=nyplo ) 
        ax = fig.add_axes( Axes , projection=projMap)
        sstlv=np.linspace(-2,2,num=11)+warming_av
        co=ax.contourf(lon,lat,warming2 ,levels=sstlv,cmap=cmap,transform=projData ,extend='both')
        cl=ax.contour( lon,lat,warming2 ,levels=sstlv ,transform=projData, colors='black')
        ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
        ax.text(-0.1, 1.1, f"{chr(97 +0)})", transform=ax.transAxes,
            fontsize=20, fontweight='bold', va='top')
        ax.set_title( SSTlab[n]  )

    cbar = plt.colorbar(co,ax=ax,shrink=0.65)
    cbar.set_label( 'K' )
    #ax.set_ylim(60,130)
    #ax.set_extent( 
    ax.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
    #ax.set_title( SSTlab[n]+ ' (LENS#'+str(nen)+')' )


In [ ]:
#############################
# Above Figure used in paper
#############################

In [ ]:
#############################
# stop
#############################

In [ ]:
##################################
# More analysis below, but nothing
# for paper ....
###################################

In [ ]:
pub_ens_x=[0,8,11,21,24,29,-1]
SSTlab = ['SST1' , 'SST2' , 'SST3' , 'SST4' , 'SST5' , 'SST6' , 'SST7' ]

projMap  = ccrs.PlateCarree(central_longitude=180.)
projData = ccrs.PlateCarree()

tsZonA=np.zeros((35,192,288))
for em in np.arange( 35):
    for y in np.arange( 192 ):
        tsZonA[em,y,:] = tsRoog[em,y,:] - tsRoog[em,y,:].mean(axis=0)

In [ ]:
fig = plt.figure(figsize=(50, 10))
sstlv=np.linspace(-.5,.5,num=11)
for n in np.arange(start=0,stop=7):
    ax=fig.add_subplot(2,4,n+1,projection=projMap )
    nen=pub_ens_x[n]
    co=ax.contourf(lon,lat,tsZonA[nen,:,:] ,levels=sstlv,cmap='bwr',transform=projData)
    cl=ax.contour( lon,lat,tsZonA[nen,:,:] ,levels=sstlv ,transform=projData)
    ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
    plt.colorbar(co,ax=ax,shrink=0.5)
     #ax.set_ylim(60,130)
    #ax.set_extent( 
    ax.set_extent( [180.-140,180+170, -50, 50] , crs=projData)
    ax.set_title( SSTlab[n]+ ' (LENS#'+str(nen)+')' )

In [ ]:
fig = plt.figure(figsize=(8, 5))
sstlv=np.linspace(-.5,.5,num=16)

"""
#d00=ts1[:,:,:].mean(axis=0)-tsCens.mean( axis=0 )
plt.contourf( d00, levels=sstlv ,cmap='jet')
plt.contour( d00, levels=sstlv )
plt.xlim(100,288)
plt.ylim(60,130)
"""

#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

fig = plt.figure(figsize=(30, 25))
sstlv=np.linspace(-.5,.5,num=16)
for nen in np.arange(start=0,stop=31):

    nxplo=5 
    nyplo=6
    dxplo=0.9/nxplo 
    dyplo=0.9/nyplo
    dxcor=1./nxplo
    dycor=0.5/nyplo
    
    jplo = (nen-1)//nxplo + 1
    iplo = (nen-1)-jplo*nxplo
    Axes = [ iplo*dxcor , 1.-jplo*dycor , dxplo , dyplo ] 
    
    ax = fig.add_axes( Axes , projection=projPC)
    #ax=fig.add_subplot(6,5,nen  , projection=projPC )

    co=ax.contourf(lon,lat, tsRoog[nen,:,:] ,levels=sstlv,cmap='jet' , transform=data_crs )
    cl=ax.contour( lon,lat,tsRoog[nen,:,:] ,levels=sstlv,  transform=data_crs )
    #ax.coastlines(resolution='110m',color='black')
    ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
    #ax.set_ylim(60,130)
    #ax.set_extent( 
    ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
    ax.set_title( '#'+str(nen) )


#projPC = ccrs.Robinson(central_longitude=180.)
projPC = ccrs.PlateCarree(central_longitude=180.)
data_crs = ccrs.PlateCarree()

ensN = [0,-1]

fig = plt.figure(figsize=(20, 12.5))
sstlv=np.linspace(-.5,.5,num=16)
for nplo in np.arange(2):

    nxplo=2 
    nyplo=1
    dxplo=0.9/nxplo 
    dyplo=0.9/nyplo
    dxcor=1./nxplo
    dycor=0.5/nyplo
    
    nen=ensN[nplo]
    
    jplo = (nplo)//nxplo + 1
    iplo = (nplo)-(jplo-1)*nxplo
    Axes = [ iplo*dxcor , 1.-jplo*dycor , dxplo , dyplo ] 
    print( Axes )
    ax = fig.add_axes( Axes , projection=projPC)
    #ax=fig.add_subplot(6,5,nen  , projection=projPC )

    co=ax.contourf(lon,lat, tsRoog[nen,:,:] ,levels=sstlv,cmap='jet' , transform=data_crs )
    cl=ax.contour( lon,lat,tsRoog[nen,:,:] ,levels=sstlv,  transform=data_crs )
    #ax.coastlines(resolution='110m',color='black')
    ax.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='black')    #ax.set_xlim(100,280)
    #ax.set_ylim(60,130)
    #ax.set_extent( 
    ax.set_extent( [180.-150,180+170, -40, 40] , crs=data_crs )
    ax.set_title( '#'+str(nen) )

cax = plt.axes([0.0, 0.8, 0.9, 0.02])
plt.colorbar(co,cax=cax,orientation='horizontal')


In [ ]:
pub_ens=[8,11,21,24,29]

In [ ]:
tsRa = np.average(np.average(np.average(tsR,axis=0), axis=0), axis=0) 
tsCa = np.average(np.average(np.average(tsC,axis=0), axis=0), axis=0) 




In [ ]:
fig = plt.figure(figsize=(20, 10))
sstlv=np.linspace(270,310,num=41)
ax=fig.add_subplot(2,1,1)
co=ax.contourf( tsRa ,levels=sstlv)
plt.colorbar( co )
ax=fig.add_subplot(2,1,2)
co=ax.contourf( tsCa ,levels=sstlv)
plt.colorbar( co )




In [ ]:
tsRjason = np.average( tsR[:,:,6:11,:,:] , axis= 2 )

In [ ]:
tsRjason_natl = np.average( np.average( tsRjason[:,:,100:140,235:250],axis=3),axis=2)
tsRjason_npac = np.average( np.average( tsRjason[:,:,100:140,100:150],axis=3),axis=2)


In [ ]:
plt.plot( (tsRjason_natl-tsRjason_npac).flatten())

In [ ]:
tsoo=np.average( ts, axis=1 )
tsoo.shape


In [ ]:
# ooooohhhh 

# 'Center' the data

In [ ]:
# Don't be bothered by apparent dimension mismatch.
# This line 'broadcasts' the mean taken for axis=0
# to the other dimensions, i.e., this line is equivalent 
# to subtracting 2 3D arrays where the mean array is 
# a stack of identical 2D arrays. Thanks ChatGPT.
#-----------------------------------------------------
tsoop = tsoo - tsoo.mean(axis=0)


In [ ]:
for em in np.arange( nens ):
    tsoop[em, :,:] = np.where( landf < 0.05 , tsoop[em,:,:], 0. )

In [ ]:
# Scale with SQRT(cos(lat))
Lon,Lat=np.meshgrid(lon,lat) 
CosScale=np.sqrt( np.cos( (np.pi/180.)*Lat ) )
print(np.shape(CosScale))
plt.contour( CosScale )

In [ ]:
# More "broadcasting"
tsoog = tsoop * CosScale

In [ ]:
#plt.plot( tsoo[:,90,90]-304.)
plt.contour( tsoog[0,:,:])


In [ ]:
print(lat[[60,131]])
tsoog=tsoog[:,60:132,:]
nt,ny,nx = np.shape(tsoog)

In [ ]:
tsoog=np.reshape( tsoog, (nt, ny*nx) )

In [ ]:
u,s,vt=np.linalg.svd( tsoog, full_matrices=False )

In [ ]:
print(np.shape(u))
print(np.shape(s))
#plt.plot(np.average(u,axis=1))
plt.plot( u.flatten() )

In [ ]:
#print( np.shape(vt))
fig=plt.figure(figsize=(20,10))

lv=np.linspace( -.03,.03,num=31)
for n in np.arange(12):
    ax1 = fig.add_subplot(3, 4, n+1 ) # , projection=projPC )
    co=ax1.contourf( vt[n,:].reshape(ny,nx) , levels=lv)
    plt.colorbar(co ) #,cax=cax,orientation='horizontal')


In [ ]:
## 

In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
pca=PCA(n_components=20)

In [ ]:
tsoov=pca.fit_transform(tsoop)

In [ ]:
print(np.shape(pca.components_))

In [ ]:
plt.plot( pca.components_[0,:])
plt.plot( vt[0,:])


In [ ]:
print(192*288)

In [ ]:
skoov=np.reshape( pca.components_ , (20,192,288) )

In [ ]:
plt.contour(skoov[14,:,:])